In [1]:
import os
import cv2
import numpy as np

In [2]:
def breaker(num: int = 50, char: str = "*") -> None:
    print("\n" + num*char + "\n")


def preprocess(image: np.ndarray, size: int) -> np.ndarray:
    return cv2.resize(src=cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB), dsize=(size, size), interpolation=cv2.INTER_AREA)


def get_images(path: str, names: np.ndarray, size: int) -> np.ndarray:
    images = np.zeros((len(names), size, size, 3), dtype=np.uint8)
    
    i = 0
    for name in names:
        images[i] = preprocess(cv2.imread(os.path.join(path, name), cv2.IMREAD_COLOR), size)
        i += 1
    return images


def save(train_images: np.ndarray, 
         test_images: np.ndarray, 
         train_targets: np.ndarray,
         test_targets: np.ndarray, 
         size: int) -> None:
    np.save(f"train_images_{size}.npy", train_images)
    np.save(f"test_images_{size}.npy", test_images)
    np.save(f"train_targets_{size}.npy", train_targets)
    np.save(f"test_targets_{size}.npy", test_targets)

    

def get_statistics(images: np.ndarray, size: int) -> None:
    print(f"Mean {size}x{size}\n")
    print(f"Red Channel Mean   {size} : {images[:, :, :, 0].mean() / 255:.5f}")
    print(f"Green Channel Mean {size} : {images[:, :, :, 1].mean() / 255:.5f}")
    print(f"Blue Channel Mean  {size} : {images[:, :, :, 2].mean() / 255:.5f}")
    print("")
    print(f"Standard Deviation {size}x{size}\n")
    print(f"Red Channel Std    {size} : {images[:, :, :, 0].std() / 255:.5f}")
    print(f"Green Channel Std  {size} : {images[:, :, :, 1].std() / 255:.5f}")
    print(f"Blue Channel Std   {size} : {images[:, :, :, 2].std() / 255:.5f}")

In [3]:
size: int = 512

train_filenames_pneumonia: list = os.listdir("../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA")
train_filenames_normal: list    = os.listdir("../input/chest-xray-pneumonia/chest_xray/train/NORMAL")

valid_filenames_pneumonia: list = os.listdir("../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA")
valid_filenames_normal: list    = os.listdir("../input/chest-xray-pneumonia/chest_xray/val/NORMAL")

test_filenames_pneumonia: list = os.listdir("../input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA")
test_filenames_normal: list    = os.listdir("../input/chest-xray-pneumonia/chest_xray/test/NORMAL")

In [4]:
train_pneumonia_images = get_images(
    "../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA", 
    train_filenames_pneumonia, 
    size
)
train_normal_images    = get_images(
    "../input/chest-xray-pneumonia/chest_xray/train/NORMAL", 
    train_filenames_normal, 
    size
)

valid_pneumonia_images = get_images(
    "../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA", 
    valid_filenames_pneumonia, 
    size
)
valid_normal_images    = get_images(
    "../input/chest-xray-pneumonia/chest_xray/val/NORMAL", 
    valid_filenames_normal, 
    size
)

test_pneumonia_images = get_images(
    "../input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA", 
    test_filenames_pneumonia, 
    size
)
test_normal_images    = get_images(
    "../input/chest-xray-pneumonia/chest_xray/test/NORMAL", 
    test_filenames_normal, 
    size
)

train_pneumonia_images = np.concatenate((train_pneumonia_images, valid_pneumonia_images), axis=0)
train_normal_images    = np.concatenate((train_normal_images, valid_normal_images), axis=0)

train_images = np.concatenate((train_pneumonia_images, train_normal_images), axis=0)
test_images  = np.concatenate((test_pneumonia_images, test_normal_images), axis=0)

train_targets = np.concatenate((np.ones(len(train_pneumonia_images, )), np.zeros(len(train_normal_images, ))), axis=0)
test_targets  = np.concatenate((np.ones(len(test_pneumonia_images, )), np.zeros(len(test_normal_images, ))), axis=0)

save(train_images, test_images, train_targets, test_targets, size)

In [5]:
def get_statistics(images: list, size: int) -> None:
    print("\n" + 50*"*" + "\n")
    print(f"Statistics {size}x{size}\n")
    
    for i in range(3):
        mean, std = 0.0, 0.0
        for j in range(len(images)):
            mean += images[j][:, :, i].mean()
            std += images[j][:, :, i].std()
        
        if i == 0:
            print(f"\nRed Channel Mean   {size} : {mean / len(images) / 255:.5f}")
            print(f"Red Channel Std    {size} : {std / len(images) / 255:.5f}")
        
        if i == 1:
            print(f"\nGreen Channel Mean {size} : {mean / len(images) / 255:.5f}")
            print(f"Green Channel Std  {size} : {std / len(images) / 255:.5f}")

        if i == 2:
            print(f"\nBlue Channel Mean  {size} : {mean / len(images) / 255:.5f}")
            print(f"Blue Channel Std   {size} : {std / len(images) / 255:.5f}")
    
    print("\n" + 50*"*" + "\n")

get_statistics(list(train_images), size)


**************************************************

Statistics 512x512


Red Channel Mean   512 : 0.48231
Red Channel Std    512 : 0.22267

Green Channel Mean 512 : 0.48231
Green Channel Std  512 : 0.22267

Blue Channel Mean  512 : 0.48231
Blue Channel Std   512 : 0.22267

**************************************************

